# Setup

In [38]:
%reload_ext autoreload

In [39]:
%cd /home/kpmurphy/github/shifty
#from shifty.skax.logreg_flax import *
from shifty.skax.skax import *

print(loglikelihood_fn) # check that one of the symbols is defiend

/home/kpmurphy/github/shifty
<function loglikelihood_fn at 0x7f5cc85f2950>


In [3]:
# Silence WARNING:root:The use of `check_types` is deprecated and does not have any effect.
# https://github.com/tensorflow/probability/issues/1523
import logging

logger = logging.getLogger()


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

In [4]:

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3)
import scipy.stats
import einops
from functools import partial

from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time
import chex
import typing
from typing import Any, Callable, Sequence

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit
from jax import lax, random, numpy as jnp
#import jax.debug

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax
import tensorflow as tf
import tensorflow_datasets as tfds

import torch
from torch.utils.data import TensorDataset, DataLoader

import distrax
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalFullCovariance as MVN

#jax.config.update("jax_enable_x64", False)


import sklearn.datasets
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression

In [7]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


# Logreg

In [ ]:
from shifty.skax.logreg_flax import *

print(loglikelihood_fn) # check that one of the symbols is defiend

In [5]:
def make_test_data(key):
    iris = sklearn.datasets.load_iris()
    X = iris["data"]
    #y = (iris["target"] == 2).astype(np.int)  # 1 if Iris-Virginica, else 0'
    y = iris["target"]
    nclasses = len(np.unique(y)) # 
    ndata, ndim = X.shape  # 150, 4
    key = jr.PRNGKey(0)
    noise = jr.normal(key, (ndata, ndim)) * 2.0
    X = X + noise # add noise to make the classes less separable
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X = np.array(X)
    perm = jr.permutation(key, ndata)
    X, y = X[perm], y[perm]
    return X, y

key = jr.PRNGKey(0)
X,y = make_test_data(key)
print(X.shape)
print(y.shape)
print(type(X))
#print(type(y))

(150, 4)
(150,)
<class 'numpy.ndarray'>


In [6]:
X = np.reshape(np.arange(10), (10,1))
y = np.arange(10)
print([X.shape, y.shape])
iterator  = make_data_iterator(X, y, 5)
for i in range(8):
    batch = next(iterator)
    print(batch['X'].shape)
    print(batch['X'])

  

[(10, 1), (10,)]


2022-10-08 17:25:41.846184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-10-08 17:25:41.858670: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)


(5, 1)
[[2]
 [1]
 [0]
 [0]
 [7]]
(5, 1)
[[6]
 [5]
 [7]
 [0]
 [4]]
(5, 1)
[[0]
 [0]
 [3]
 [9]
 [8]]
(5, 1)
[[6]
 [0]
 [2]
 [7]
 [3]]
(5, 1)
[[3]
 [5]
 [0]
 [9]
 [5]]
(5, 1)
[[2]
 [7]
 [6]
 [1]
 [8]]
(5, 1)
[[2]
 [6]
 [7]
 [2]
 [9]]
(5, 1)
[[8]
 [4]
 [8]
 [7]
 [1]]


In [10]:
X,y = make_test_data()
# We set C to a large number to turn off regularization.
# We don't fit the bias term to simplify the comparison below.
l2reg = 1e-2
log_reg = LogisticRegression(solver="lbfgs", C=1/l2reg, fit_intercept=True)
log_reg.fit(X, y)
W_mle = log_reg.coef_ # (nclasses, ndim)
b_mle = log_reg.intercept_ # (nclasses,)
true_probs = log_reg.predict_proba(X)
nclasses, ndim = W_mle.shape
ntrain = X.shape[0]



In [35]:
key = jr.PRNGKey(0)
nclasses, ndim = W_mle.shape
print([nclasses, ndim])
key = jr.PRNGKey(0)
model = LogReg(key,  nclasses, W_init=W_mle.T, b_init=b_mle, l2reg=l2reg)
probs = np.array(model.predict(X))

model = LogReg(key, nclasses)
model.fit(X, y)
probs = np.array(model.predict(X))

print('pred: ', probs[0])
print('truth: ', true_probs[0])


[3, 4]


/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


pred:  [0.851 0.14  0.009]
truth:  [0.858 0.134 0.008]


In [36]:
def compare_method(optimizer, name=None, batch_size=ntrain, max_iter=500):
    key = jr.PRNGKey(0)
    model = LogReg(key, nclasses, max_iter=max_iter, l2reg=l2reg, optimizer=optimizer, batch_size=batch_size)  
    model.fit(X, y)
    probs = np.array(model.predict(X))
    print('method {:s}, max deviation from true probs {:.3f}'.format(name, np.max(true_probs - probs)))
    print('truth: ', true_probs[0])
    print('pred: ', probs[0])

In [37]:
compare_method("LBFGS", "LBFGS")

method LBFGS, max deviation from true probs 0.011
truth:  [0.858 0.134 0.008]
pred:  [0.855 0.137 0.009]


In [38]:
compare_method(optax.adam(0.01), name="adam 0.01, bs=N", batch_size=ntrain)

method adam 0.01, bs=N, max deviation from true probs 0.163
truth:  [0.858 0.134 0.008]
pred:  [0.801 0.169 0.03 ]


In [39]:
compare_method(optax.adam(0.01), name="adam 0.01, bs=32", batch_size=32)

method adam 0.01, bs=32, max deviation from true probs 0.235
truth:  [0.858 0.134 0.008]
pred:  [0.771 0.185 0.044]


In [40]:
compare_method("polyak", name="polyak, bs=32", batch_size=32)

method polyak, bs=32, max deviation from true probs 0.645
truth:  [0.858 0.134 0.008]
pred:  [0.735 0.256 0.009]


In [41]:
compare_method("armijo", name="armijo, bs=32", batch_size=32)

method armijo, bs=32, max deviation from true probs 0.251
truth:  [0.858 0.134 0.008]
pred:  [0.757 0.199 0.044]


# Logreg pipeline

In [43]:

import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression

def fit_classifier_sklearn(key, X, Y):
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogisticRegression(random_state=0, max_iter=100, C=1e5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
clf = fit_classifier_sklearn(key, X, y)

true_probs = clf.predict_proba(X)
print(true_probs[0])
print(np.sum(true_probs[0]))

[8.617e-01 1.380e-01 2.624e-04]
1.0000000000000002


In [46]:

def fit_classifier_skax(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogReg(key, nclasses, max_iter=500, l2reg=1e-5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs - probs)))
print('truth: ', true_probs[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.006
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [8.606e-01 1.391e-01 2.525e-04]


# Skax logreg

In [54]:
def print_probs(probs):
    str = ['{:0.3f}'.format(p) for p in probs]
    print(str)


In [14]:
def make_iris_data(key):
    iris = sklearn.datasets.load_iris()
    X = iris["data"]
    #y = (iris["target"] == 2).astype(np.int)  # 1 if Iris-Virginica, else 0'
    y = iris["target"]
    nclasses = len(np.unique(y)) # 
    ndata, ndim = X.shape  # 150, 4
    key = jr.PRNGKey(0)
    noise = jr.normal(key, (ndata, ndim)) * 2.0
    X = X + noise # add noise to make the classes less separable
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X = np.array(X)
    perm = jr.permutation(key, ndata)
    X, y = X[perm], y[perm]
    return X, y

key = jr.PRNGKey(0)
X,y = make_iris_data(key)

ninputs = X.shape[1]
ntrain = X.shape[0]
nclasses = len(np.unique(y))

In [62]:
def make_random_data(seed):
    X, y = sklearn.datasets.make_classification(n_samples=500, n_features=20,  n_informative=10, n_redundant=5, n_repeated=0,
    n_classes=10, n_clusters_per_class=1, weights=None, flip_y=0.01, class_sep=0.5,
    hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=seed)
    return X, y

X,y = make_random_data(0)
nfeatures = X.shape[1]
ntrain = X.shape[0]
nclasses = len(np.unique(y))
print([ntrain, nfeatures, nclasses])

[500, 20, 10]


In [51]:

l2reg = 1e-2
log_reg = LogisticRegression(solver="lbfgs", C=1/l2reg)
log_reg.fit(X, y)
true_probs = log_reg.predict_proba(X)


/home/kpmurphy/mambaforge/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [63]:
classifier = Pipeline([
            ('standardscaler', StandardScaler()), 
            ('logreg', LogisticRegression(random_state=0, max_iter=100, C=1/l2reg))])
classifier.fit(X, y)
true_probs = classifier.predict_proba(X)
print_probs(true_probs[0])


['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']


In [64]:
def compare_skax_method(optimizer, name=None, batch_size=ntrain, max_iter=500, num_epochs=5):
    key = jr.PRNGKey(0)
    network = LogRegNetwork(nclasses = nclasses)
    model = NeuralNetClassifier(network, key, nclasses, l2reg=l2reg, optimizer = optimizer, batch_size=batch_size,
                max_iter=max_iter, num_epochs=num_epochs, print_every=5) 
    model.fit(X, y)
    probs = np.array(model.predict(X))
    print('method {:s}, max deviation from true probs {:.3f}'.format(name, np.max(true_probs - probs)))
    print('truth'); print_probs(true_probs[0])
    print('pred'); print_probs(probs[0])

In [65]:
compare_skax_method("LBFGS", "LBFGS")

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


method LBFGS, max deviation from true probs 0.002
truth
['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']
pred
['0.215', '0.030', '0.100', '0.043', '0.017', '0.048', '0.066', '0.086', '0.002', '0.394']


In [66]:
compare_skax_method("armijo", "armijo", batch_size=ntrain)

method armijo, max deviation from true probs 0.105
truth
['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']
pred
['0.250', '0.023', '0.098', '0.052', '0.015', '0.050', '0.072', '0.067', '0.003', '0.371']


In [67]:
compare_skax_method("armijo", "armijo", batch_size=32)

method armijo, max deviation from true probs 0.385
truth
['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']
pred
['0.124', '0.021', '0.082', '0.056', '0.015', '0.041', '0.061', '0.100', '0.001', '0.499']


In [68]:

compare_skax_method(optax.adam(1e-3), name="adam", batch_size=32, num_epochs=50)

epoch 0, train loss 60.227
epoch 5, train loss 57.272
epoch 10, train loss 55.068
epoch 15, train loss 53.719
epoch 20, train loss 52.199
epoch 25, train loss 51.313
epoch 30, train loss 50.806
epoch 35, train loss 50.197
epoch 40, train loss 49.542
epoch 45, train loss 49.370
method adam, max deviation from true probs 0.376
truth
['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']
pred
['0.140', '0.039', '0.116', '0.067', '0.029', '0.083', '0.139', '0.093', '0.009', '0.285']


In [69]:
batch_size = 32
nepochs = 50
total_steps = nepochs*(ntrain//batch_size) 
print(total_steps)
warmup_cosine_decay_scheduler = optax.warmup_cosine_decay_schedule(init_value=1e-3, peak_value=1e-1,
                                                                   warmup_steps=int(total_steps*0.1),
                                                                   decay_steps=total_steps, end_value=1e-3)
optimizer = optax.adam(learning_rate=warmup_cosine_decay_scheduler)
compare_skax_method(optimizer, name="adam with warmup, bs=32", batch_size=batch_size, num_epochs=nepochs)

750
epoch 0, train loss 58.996
epoch 5, train loss 51.444
epoch 10, train loss 52.099
epoch 15, train loss 51.191
epoch 20, train loss 50.563
epoch 25, train loss 49.535
epoch 30, train loss 49.381
epoch 35, train loss 48.599
epoch 40, train loss 48.014
epoch 45, train loss 48.174
method adam with warmup, bs=32, max deviation from true probs 0.038
truth
['0.213', '0.030', '0.100', '0.043', '0.017', '0.049', '0.065', '0.085', '0.002', '0.396']
pred
['0.208', '0.030', '0.105', '0.043', '0.017', '0.046', '0.063', '0.076', '0.002', '0.409']


# Neural nets with no hidden layers

In [160]:
def fit_classifier_sklearn(key, X, Y):
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogisticRegression(random_state=0, max_iter=100, C=1e5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
clf = fit_classifier_sklearn(key, X, y)

true_probs_pipeline = clf.predict_proba(X)
print(true_probs_pipeline[0])
print(np.sum(true_probs_pipeline[0]))

[8.617e-01 1.380e-01 2.624e-04]
1.0000000000000002


In [161]:
def fit_classifier_skax_mlp(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    network = MLPNetwork((nclasses,))
    model = NeuralNetClassifier(network, key, nclasses, l2reg=1e-5, max_iter=500)
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', model)])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax_mlp(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs_pipeline - probs)))
print('truth: ', true_probs_pipeline[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.004
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [8.622e-01 1.375e-01 2.553e-04]


In [156]:
compare_mlp_method("LBFGS", "LBFGS")

fit ninputs  4
method LBFGS, max deviation from true probs 0.011
truth:  [0.858 0.134 0.008]
pred:  [0.855 0.137 0.009]


In [157]:

compare_skax_method("armijo", name="armijo, bs=32", batch_size=32)

fit ninputs  4
method armijo, bs=32, max deviation from true probs 0.251
truth:  [0.858 0.134 0.008]
pred:  [0.757 0.199 0.044]


# Neural nets with hidden layers

In [162]:
def fit_classifier_skax_mlp(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    network = MLPNetwork((5, nclasses,))
    model = NeuralNetClassifier(network, key, nclasses, l2reg=1e-5, max_iter=500)
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', model)])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax_mlp(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs_pipeline - probs)))
print('truth: ', true_probs[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.934
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [1.00e+00 8.77e-26 0.00e+00]
